# **Paso 1: Datos Crudos de Ensayo Clínico**

`Dataset: (NCT00174655)`

- Fase III sobre cáncer de mama
- Participaron 2,887 pacientes
- Quienes fueron asignados al azar a distintos tratamientos para evaluar la eficacia del medicamento Docetaxel (usado solo o con Doxorubicina), seguido de CMF

`feature.csv`
Características estáticas de los pacientes (peso, altura, etc.)
![Figure 1](1.png)

`visit.pkl`
Matriz. Cada lista interior representa a un paciente, y dentro de ella, cada elemento es una visita. A su vez, cada visita es una lista que contiene los eventos ocurridos.
![Figure 3](3.png)

`voc.pkl`
Contiene vocabularios que convierten los nombres de los eventos (como "dolor de cabeza") en números enteros. Por ejemplo, {'dolor de cabeza': 1, 'náuseas': 2, ...}
![Figure 4](4.png)

# **Paso 2: Estructuración en Secuencias por Paciente**

`TWIN.ipynb`

In [ ]:
# Cargar datos de demostración del ensayo clínico
data = load_trial_patient_sequence()
print("Datos cargados:")
for key, value in data.items():
    if hasattr(value, 'shape'):
        print(f"  {key}: {value.shape}")
    elif hasattr(value, '__len__'):
        print(f"  {key}: {len(value)} elementos")
    else:
        print(f"  {key}: {type(value)}")
data

`load_trial_patient_sequence()` en `data/demo.py`

In [ ]:
def load_trial_patient_sequence(input_dir=None):

    print("#"*5+'Demo Data Folder'+"#"*5)
    print(os.listdir(input_dir))
    print("#"*20)

    # 1. Carga de archivos
    visit = dill.load(open(os.path.join(input_dir,'visit.pkl'), 'rb')) # Matriz. Cada lista interior es un paciente y dentro de cada paciente hay una lista de visitas. Cada visita es una lista que contiene los eventos ocurridos.
    vocs = dill.load(open(os.path.join(input_dir,'voc.pkl'), 'rb')) #Diccionario traductor. convierten los nombres de los eventos (como "dolor de cabeza") en números enteros. {'dolor de cabeza': 1, 'náuseas': 2, ...}
    feature = pd.read_csv(os.path.join(input_dir, 'feature.csv')) #Características estáticas de los pacientes (peso, altura, etc.).
    v_stage = dill.load(open(os.path.join(input_dir,'visit_stage.pkl'), 'rb')) #Representa las etapas de la enfermedad de un paciente en cada visita.Por ejemplo, si un paciente tiene 5 visitas registradas en visit.pkl, visit_stage contendría una secuencia de 5 elementos indicando el estadio de la enfermedad en cada una de esas visitas.
    orders = list(vocs.keys())
    
    # 2. Procesamiento de Características Estáticas (de feature.csv)
    label_relapse = feature['num relapse']
    label_mortality = feature['death'].values
    x = feature.drop(['num relapse','death','RUSUBJID'], axis=1) #caracteristicas estaticas de la persona
    x['weight'] = x['weight'].replace({'>= 125':'125'}).astype(float) #Reemplaza el texto '>= 125' por el número '125', tratando los valores de peso por encima de 125 como si fueran 125.

    # 3. Procesamiento de Características Estáticas (de feature.csv)
    tabx = TabularPatientBase(x)

    x = tabx.df.values # get processed patient features in matrix form
    return {
        'feature':x,
        'visit':visit,
        'voc':vocs,
        'order':orders,
        'visit_stage':v_stage,
        'relapse':label_relapse,
        'mortality':label_mortality,
    } 

In [ ]:
 'relapse': 0      0
            1      0
            2      1
            3      0
            4      0
                ..
            972    0
            973    1
            974    1
            975    0
            976    0

In [ ]:
'mortality': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        ...)}

`TabularPatientBase(x)` en `data/patient_data.py`

In [ ]:
class TabularPatientBase(Dataset):
    def __init__(self, df, metadata=None, transform=True):
        self.df = df
        self.metadata = metadata

        # initialize hypertransformer
        self.ht = HyperTransformer()

        if transform:
            if metadata is None:
                warnings.warn('No metadata provided. Metadata will be automatically '
                            'detected from your data. This process may not be accurate. '
                            'We recommend writing metadata to ensure correct data handling.')
                self.ht.detect_initial_config(df)
                self.metadata = self.ht.get_config()
                self.ht.fit(df)

            else:
                # parse the metadata and update hypertransformer's config
                self._parse_metadata()
            
            # create a mapping from column name before to column name after transformation
            self._create_transformed_col2col()

            # replace data with the transformed one
            self.df = self.transform(df)

`HyperTransformer()` en `utils/tabular_utils.py`

In [ ]:
class HyperTransformer(rdt.HyperTransformer):
    '''
    A subclass of `rdt.HyperTransformer` to set special setups.
    '''
    _DTYPES_TO_SDTYPES = {
        'i': 'categorical', # change the default from numerical to categorical for integers
        'f': 'numerical',
        'O': 'categorical',
        'b': 'boolean',
        'M': 'datetime',
    }
    def __init__(self):
        self._default_sdtype_transformers = {
            'numerical': StandardScaler(missing_value_replacement='mean'),
            'categorical': LabelEncoder(),
            'boolean': BinaryEncoder(missing_value_replacement='mode'),
            'datetime': UnixTimestampEncoder(missing_value_replacement='mean'),
        }
    ...


### Columnas Categóricas: `LabelEncoder`

**Para qué sirve:** Los modelos matemáticos no entienden de texto como "Tipo A" o "Tipo B". Necesitan números. El `LabelEncoder` asigna un número entero único a cada categoría de texto.

**Ejemplo con la columna `blood_type`:** `['A', 'O', 'A', 'AB', 'B']`

*   **Paso 1: Asignar un número entero a cada una**
    *   'A' -> **0**
    *   'AB' -> **1**
    *   'B' -> **2**
    *   'O' -> **3**
    *(El orden de la asignación puede variar, pero será consistente).*

---

### Columnas Booleanas: `BinaryEncoder`

*   **Paso 1: Definir el mapeo**
    *   `True` -> **1**
    *   `False` -> **0**

---

### DataFrame Final Transformado


Digamos que nuestro DataFrame `x` original es así:


| weight | blood_type | high_risk |
| :----: | :---------:|:---------:|
|    65  |      'A'   |    True|
|   90   |     'O'    |  False |
|  75    |    'A'     |  True |
| 90     |  'AB'     |  True |
|80      |  'B'     | False |


Después de que `tabx = TabularPatientBase(x)` hace su trabajo, el DataFrame que contiene (`tabx.df`) se vería así:

| weight_transformed | blood_type_encoded | high_risk_encoded |
| :-----------------: | :------------------: | :-----------------: |
|        -1.56        |           0          |          1          |
|         1.04        |           3          |          0          |
|        -0.52        |           0          |          1          |
|         1.04        |           1          |          1          |
|         0.00        |           2          |          0          |

Este DataFrame es **puramente numérico y está estandarizado**, listo para ser procesado por las capas de una red neuronal o cualquier otro modelo de machine learning.

# **Paso 3: Creación del Objeto SequencePatient**


`TWIN.ipynb`

In [ ]:
# Crear objeto SequencePatient con los datos cargados
seqdata = SequencePatient(
    data={
        "v": data["visit"],
        "y": data["mortality"],
    },
    metadata={
        "voc": data["voc"],
        "order": data["order"]
    }
)

print(f"Datos de secuencia creados con {len(seqdata)} pacientes")


El Paso 3 **NO cambia los datos**, solo los **reorganiza** para que el modelo TWIN pueda usarlos fácilmente.

#### **ANTES** - Datos separados y desordenados:
```python
 data = {
    'visit': [
        [[[0,1], [15,17], [45,67]], ... ], # Paciente 0: tratamiento, medicamento, efecto adverso
        [[[1], [20], [50]]]  # Paciente 1: tratamiento, medicamento, efecto adverso
    ],
    'mortality': [0, 1],     # ¿Murió? Paciente 0: No, Paciente 1: Sí
    'voc': {...}             # Diccionarios separados
}

# Para obtener info del primer paciente necesitas:
visitas_paciente_0 = data['visit'][0]     # Buscar en 'visit'
mortalidad_paciente_0 = data['mortality'][0] # Buscar en 'mortality'
vocabulario = data['voc']                  # Buscar en 'voc'
```

#### **DESPUÉS** - Todo organizado en un solo objeto:
```python
 seqdata = SequencePatient(...)

# Para obtener info del primer paciente:
index, visitas, mortalidad, orden, _ = seqdata[0]  # ✨ Todo en una línea

```

# **Paso 4: Conversión a Vectores Multi-hot**

Imagina que tienes un vocabulario de medicamentos:
```
Vocabulario de medicamentos:
0: Aspirina
1: Ibuprofeno  
2: Paracetamol
3: Antibiótico
4: Insulina
```

**Vector multi-hot** significa: un vector donde cada posición representa si ese medicamento está presente (1) o ausente (0).

### **Ejemplo:**

**ANTES (códigos densos):**
```python
# Un paciente en una visita tomó:
medications_taken = [1, 3]  # Ibuprofeno (índice 1) y Antibiótico (índice 3)
```

**DESPUÉS (vector multi-hot):**
```python
# Vector de tamaño 5 (total de medicamentos en vocabulario)
medication_multihot = [0, 1, 0, 1, 0]
#                     ^  ^  ^  ^  ^
#                     |  |  |  |  └── Insulina (NO tomada)
#                     |  |  |  └────── Antibiótico (SÍ tomada) 
#                     |  |  └──────── Paracetamol (NO tomada)
#                     |  └─────────── Ibuprofeno (SÍ tomada)
#                     └────────────── Aspirina (NO tomada)
```
---

`TWIN.ipynb`

```python
# Inicializar y entrenar el modelo TWIN
model = TWIN(
    epochs=20, 
    vocab_size=vocab_size,
    order=["treatment", "medication", "adverse_event"],
    freeze_event=["treatment"],  # Mantener tratamientos fijos durante la generación
    device='cpu',  # 'cuda:0' o cpu
    batch_size=300,
    verbose=True,
    k=5 # número de vecinos para el retrieval
)

print("Iniciando entrenamiento...")
model.fit(seqdata)
```

` TWIN class en twin.py`

```python
class TWIN(SequenceSimulationBase):
    ...

    def fit(self, train_data):
        '''
        Fit the model with training data.

        Parameters
        ----------
        train_data: SequencePatientBase
            Training data.
        '''
        self._input_data_check(train_data)
        df_train_data = self._translate_sequence_to_df(train_data) <---

        # train the model for each event type
        for et in self.config['perturb_event']:
            model = self.models[et]
            model._fit_model(df_train_data)
        
        if self.config["verbose"]:
            print("Training finished.")

```

`_translate_sequence_to_df() de TWIN class`

```python
# En _translate_sequence_to_df()
for k in range(len(self.config["orders"])):  # k=0,1,2 (treatment, medication, adverse_event)
    event_binary = np.array([0]*self.config['vocab_size'][k])
    event_binary[inputs[i][j][k]] = 1
```

---

### **DESPUÉS de la conversión (DataFrame final):**

```python
# El DataFrame resultante tiene columnas así:
column_names = ['People', 'Visit', 
               'treatment_0', 'treatment_1', 'treatment_2', 'treatment_3', 'treatment_4',
               'medication_0', 'medication_1', 'medication_2', ..., 'medication_100',
               'adverse_event_0', 'adverse_event_1', ..., 'adverse_event_277']

# Una fila del DataFrame para patient_0_visit_1:
row = [0, 1,  # People=0, Visit=1
       1, 0, 0, 0, 0,  # treatment multi-hot: solo treatment_0 activo
       0, 1, 0, 0, 0, 1, 0, 0, ..., 0,  # medication multi-hot: medication_1 y medication_5 activos
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, ..., 0]  # adverse_event multi-hot
```

## **Paso 5: Preparación para Entrenamiento**

### **Función `_next_step_df()` explicada:**

Esta función prepara los datos para entrenamiento con causalidad:

### **Datos Antes:**

![Figure 5](5.png)

---

`_next_step_df()` en `/trial_simulation/twin.py - UnimodalTWIN()`

```python
# PROCESO: Se crean columnas "next step" para causalidad
# adverse_event en timestep t+1 depende de medication en timestep t
nxt_data = df_before[adverse_event_columns].shift(-1)  # Mover una fila hacia arriba
nxt_data.columns = ['###nxt###_adverse_event_10', '###nxt###_adverse_event_15']

# SALIDA: X (medication actual) e y (adverse_events siguiente)
X = df_before[medication_columns]  # Lo que el modelo ve
y = nxt_data  # Lo que debe predecir
```

---

### **Datos Después:**

#### *Tabla **X***
![Figure 6](6.png)

#### *Tabla **Y***
![Figure 7](7.png)

# **Paso 6: Conversión a Embeddings Densos**

### **Vector multihot - Desventajas**

```python
# Un vector de tamaño 101, casi todo en ceros.
x_multihot = [0, 1, 0, 0, 0, 1, 0, ..., 0]
#             ^  ^           ^
# Posición:   0  1           5
```
"Ibuprofeno" (índice 1) y "Antibiótico" (índice 5)

- Vectores dispersos
- No capturan ninguna relación semántica entre los eventos


### **Matriz de Embedding W_emb (La Solución)**

- Actúa como una tabla de consulta.
- Fila = "Embedding" que representa a un único evento de nuestro vocabulario
- Forma de la Matriz: [vocab_size x hidden_dim]

```python
# W_emb es una matriz entrenable de forma [101, 64]
# Cada fila es un vector de 64 números que representa un medicamento.
W_emb = [
    [0.1, -0.2, 0.8, ..., 0.3],  # Embedding para medicamento_0
    [0.5,  0.7, -0.1, ..., 0.9], # Embedding para medicamento_1 (Ibuprofeno) <--
    [0.2,  0.3,  0.4, ..., 0.1],  # Embedding para medicamento_2
    [0.8, -0.3,  0.6, ..., 0.2],  # Embedding para medicamento_3
    [0.4,  0.1, -0.5, ..., 0.7],  # Embedding para medicamento_4
    [-0.1, 0.9,  0.2, ..., 0.4],  # Embedding para medicamento_5 (Antibiótico) <--
    # ... 95 filas más
]
```

### **La Operación: h = x · W_emb**

*Según el paper: Ecuación (4)*

![Figure 8](8.png)

```python
# Se suman los embeddings de los medicamentos tomados (índices 1 y 5)
embedding_medicamento_1 = [0.5,  0.7, -0.1, ..., 0.9]
embedding_medicamento_5 = [-0.1, 0.9,  0.2, ..., 0.4]

h_dense = embedding_medicamento_1 + embedding_medicamento_5
```

### **La Salida: Embedding Denso `h` (El Resultado Deseado)**

```python
# Vector resultante, de tamaño 64.
# Es una representación combinada de "Ibuprofeno" y "Antibiótico".
h_dense = [0.4, 1.6, 0.1, ..., 1.3]
```

- Denso (lleno de información), de menor tamaño
- Captura relaciones semánticas.

### **Implementación**

`trial_simulation/model.py`

#### **Definición (La Matriz de Embedding)**

En la clase `Encoder`, la matriz W_emb se crea en la inicializacion:

```python

// ... existing code ...
class Encoder(nn.Module):
// ... existing code ...
    def __init__(self, vocab_size, event_order, freeze_order, hidden_dim, latent_dim):
// ... existing code ...
        # W_emb: Trainable embedding matrix as described in TWIN paper Equation (4)
        # This converts multi-hot vector x^u_{n,t} ∈ {0,1}^l to dense embedding h^u_{n,t} ∈ R^d
        # W_emb^u ∈ R^{l×d} where l is vocab size and d is hidden_dim
        self.W_emb = nn.Linear(input_dim, hidden_dim, bias=False)
// ... existing code ...

```

#### **La Operación (La Conversión)**

Dentro de la misma clase `Encoder`, la conversión se realiza en el método `forward`:

```python

// ... existing code ...
    def forward(self, x):
// ... existing code ...
        # Step 1: Convert multi-hot to dense embedding using trainable matrix W_emb
        # This implements Equation (4): h^u_{n,t} = x^u_{n,t} · W^u_{emb}
        h_emb = self.W_emb(x)  # Multi-hot to dense embedding conversion
        h_activated = self.LeakyReLU(h_emb)
// ... existing code ...

```

# Paso 7: **Codificación Aumentada por Recuperación (Retrieval-Augmented Encoding)**

![Figure 9](9.png)

- Buscamos las K visitas más parecidas en todo el conjunto de datos y las usamos para añadir contexto
- Evita overfitting
- Da un contexto más amplio de patrones clínicos similares.

### **La Entrada: Embedding Denso y un "Banco de Memoria"**

#### **Embedding Actual (`z`)**

Este es el vector que representa la visita que estamos analizando. Por ejemplo, la **visita 2** del **paciente 15**:

```python
# Embedding denso para la visita actual (paciente_15, visita_2)
# Este vector 'z' se calculó en el Paso 6
z = [0.4, 1.6, 0.1, 1.3]
```

#### **Banco de Memoria (`memory_bank`)**

Esta es una gran "tabla" (o matriz) donde cada fila es el embedding denso de una visita del conjunto de datos de entrenamiento.

```python
# El memory_bank contiene los embeddings de todas las visitas de entrenamiento
memory_bank = [
# ------------------------------------------------------------------ Paciente 0
    [0.1, -0.2, 0.8, 0.3],  # <- Embedding de (paciente_0, visita_0)
    [0.5,  0.7, -0.1, 0.9],  # <- Embedding de (paciente_0, visita_1)
    [0.2,  0.3,  0.4, 0.1],  # <- Embedding de (paciente_0, visita_2)
# ------------------------------------------------------------------ Paciente 1
    [-0.1, 0.9,  0.2, 0.4],  # <- Embedding de (paciente_1, visita_0)
# ------------------------------------------------------------------ ... etc.
    # ... muchísimas más filas, una por cada visita en el dataset
    [0.8, -0.3,  0.6, 0.2],
    [0.4,  0.1, -0.5, 0.7],
]
```
